# Cargar librerias

In [52]:
import pandas as pd

from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from google.colab import drive

# Montar drive

In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


# Cargar dataset

In [45]:
df = pd.read_csv("/content/drive/MyDrive/Inteligencia Artificial I/Proyecto/histogram_features_HSV.csv")
df_rp = df[(df['color'] == 'Red') | (df['color'] == 'Purple')]
df_rp

,id,color,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,...,feature_86,feature_87,feature_88,feature_89,feature_90,feature_91,feature_92,feature_93,feature_94,feature_95
87,1278,Purple,0.978236,0.000000,0.000000,0.000698,0.000000,0.017792,0.000000,0.000000,...,0.000610,0.000305,0.000610,0.000305,0.000000,0.001524,0.002744,0.002744,0.003658,0.996316
88,1378,Purple,0.973007,0.000000,0.000000,0.000000,0.000000,0.009623,0.000000,0.000000,...,0.000657,0.005258,0.001643,0.001314,0.000986,0.003286,0.000657,0.002300,0.002629,0.992741
89,1445,Purple,0.915673,0.000000,0.000000,0.000435,0.000000,0.013479,0.000000,0.000000,...,0.003061,0.002721,0.001020,0.000000,0.000680,0.002381,0.001360,0.002381,0.002721,0.994509
90,1506,Purple,0.898388,0.000960,0.006242,0.007683,0.002881,0.067223,0.000480,0.000000,...,0.004986,0.003656,0.004653,0.000997,0.001330,0.001662,0.001994,0.001330,0.003324,0.990514
91,1563,Purple,0.945461,0.000000,0.000000,0.002648,0.000883,0.067974,0.003973,0.000441,...,0.001687,0.000675,0.000337,0.002699,0.000337,0.002362,0.002699,0.002024,0.002362,0.987806
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
292,576,Red,0.961267,0.026736,0.020155,0.015219,0.011517,0.082676,0.011517,0.004113,...,0.017177,0.007928,0.008258,0.004955,0.006276,0.002312,0.001982,0.001321,0.000661,0.995959
293,914,Red,0.931572,0.034923,0.044447,0.011339,0.015420,0.126991,0.008617,0.002268,...,0.022012,0.015171,0.034804,0.038671,0.022607,0.008032,0.004165,0.005949,0.002677,0.997700
294,812,Red,0.847960,0.002803,0.007008,0.003270,0.000000,0.021024,0.000000,0.000000,...,0.000324,0.000648,0.000648,0.000648,0.000972,0.000324,0.000324,0.000324,0.001296,0.992945
295,70,Red,0.885628,0.027312,0.033382,0.015174,0.022254,0.196244,0.019220,0.003035,...,0.014099,0.014427,0.012459,0.016722,0.019673,0.024591,0.055084,0.082626,0.074757,0.991180


# Grupos por color

In [20]:
df.groupby("color")["color"].count()

,color
color,
Beige,87
Black,315
Blue,115
Brown,295
Green,87
Grey,83
Pink,92
Purple,105
Red,105


# Gaussian Naive Bayes

In [95]:
df_rp = df[(df['color'] == 'Red') | (df['color'] == 'Purple')]

# Convertir categorias a numeros
df_rp["color"] = pd.factorize(df_rp["color"])[0]

# Dividir datos
X = df_rp.iloc[:, 2:]
X["id"] = df_rp["id"]
Y = df_rp.iloc[:, 1]

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=21)

# Estimador
est = GaussianNB()
est.fit(X_train.iloc[:, :-1], y_train)
predictions = est.predict(X_test.iloc[:, :-1])

print("y_test")
print(y_test)
print("Predicciones")
print(predictions[:])

# Calcular precisión
print("Acuraccy:", accuracy_score(y_test, predictions))




y_test
139    0
104    0
127    0
93     0
96     0
279    1
191    0
237    1
287    1
216    1
141    0
210    1
157    0
107    0
170    0
239    1
198    1
231    1
285    1
89     0
232    1
193    1
290    1
100    0
172    0
108    0
236    1
132    0
201    1
293    1
218    1
202    1
175    0
229    1
162    0
149    0
106    0
244    1
283    1
109    0
252    1
249    1
Name: color, dtype: int64
Predicciones
[0 0 0 0 0 1 0 1 1 0 0 1 0 0 0 1 0 1 1 0 1 1 1 0 0 0 1 0 1 1 1 1 0 1 0 0 0
 1 1 0 1 1]
Acuraccy: 0.9523809523809523


/tmp/ipython-input-3934166680.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_rp["color"] = pd.factorize(df_rp["color"])[0]
